In [1]:
input = open("inputs/5").read()

In [2]:
test_input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [3]:
def parse_input(input):
    seed_str, *rest = input.split("\n\n")

    seeds = [int(s) for s in seed_str.split()[1:]]

    maps = []
    for map in rest:
        map_ranges = []
        lines = map.splitlines()[1:]

        for line in lines:
            map_ranges.append([int(n) for n in line.split()])

        maps.append(map_ranges)
    
    return seeds, maps

In [4]:
def p1(input):
    seeds, maps = parse_input(input)

    map_ranges = []

    for map in maps:
        ranges = []

        for (dest_start, source_start, range) in map:
            # ok so we want a mapping from source to destination number which is valid in given range of the source number
            # simplest format for my brain is to specify a range of [start, end) where the source gets transformed and then a constant representing the transformation
            # so, let's store as triple [start, end, constant]

            ranges.append([source_start, source_start + range, dest_start - source_start])
        
        map_ranges.append(ranges)


    # now let's just follow the map

    final_locations = []

    for seed in seeds:
        location = seed

        for map in map_ranges:
            for (start, end, constant) in map:
                if start <= location < end:
                    location += constant
                    break
            else:
                # location stays the same since it doesn't get modified
                pass

        final_locations.append(location)
    
    return min(final_locations)

In [5]:
p1(test_input)

35

In [6]:
p1(input)

1181555926

In [13]:
import math
from tqdm import tqdm
from itertools import batched

def p2(input):
    seeds, maps = parse_input(input)

    # pair into tuples of two
    seed_ranges = list(batched(seeds, 2))

    map_ranges = []

    for map in maps:
        ranges = []

        for (dest_start, source_start, rng) in map:
            # ok so we want a mapping from source to destination number which is valid in given range of the source number
            # simplest format for my brain is to specify a range of [start, end) where the source gets transformed and then a constant representing the transformation
            # so, let's store as triple [start, end, constant]

            ranges.append([source_start, source_start + rng, dest_start - source_start])
        
        map_ranges.append(ranges)


    # now let's just follow the map
    min_location = math.inf

    for seed_start, seed_range in seed_ranges:
        for seed in tqdm(range(seed_start, seed_start + seed_range)):
            location = seed

            for map in map_ranges:
                for (start, end, constant) in map:
                    if start <= location < end:
                        location += constant
                        break
                else:
                    # location stays the same since it doesn't get modified
                    pass
            
            if location < min_location:
                min_location = location
        

    return min_location

In [14]:
p2(test_input)

100%|██████████| 13/13 [00:00<00:00, 252434.96it/s]


46

In [15]:
p2(input)

100%|██████████| 203929368/203929368 [08:07<00:00, 418656.25it/s]


37806486

In [ ]:
# OK I'm seeing the trick. The composition of piecewise linear functions is piecewise linear. So, we can just figure out the "critical points" of the resulting composition and then only check those points. The minimum will occur at one of those.